### Convert Natural Language questions to Prolog KB with Facts and Rules

In [ ]:
import os

api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI
client = OpenAI(
    api_key=api_key
)   

In [20]:
text_prompt = '''Convert this natural language input into prolog facts and rules. Give only the code in prolog syntax that can be explicitly converted to .pl file and Do not add any comments or explanations:
Joey is Chandler's friend and Chandler is Joey's friend, who is the best friend of Joey?
Mike is married to Phoebe and Mike is male while Phoebe is female, who is the wife of Mike?
Ross and Monica are siblings and Ross is male, who is Monica's brother?
Monica is Rachel's friend, who is Monica friends with?'''

text_response = client.responses.create(
    model="gpt-4.1",
    input=text_prompt
).output_text

print(text_response)

friend(joey, chandler).
friend(chandler, joey).
best_friend(joey, chandler).

married(mike, phoebe).
male(mike).
female(phoebe).
wife(X, Y) :- married(Y, X), female(X).

sibling(ross, monica).
sibling(monica, ross).
male(ross).
brother(X, Y) :- sibling(X, Y), male(X).

friend(monica, rachel).
friends_with(X, Y) :- friend(X, Y).


### Running Prolog program to infer the answer

In [21]:
with open("output.pl", "w") as file:
    file.write(text_response)


In [22]:
from pyswip import Prolog

prolog = Prolog()
prolog.consult("output.pl")

In [23]:
queries = [
    "best_friend(joey, X)",
    "best_friend(joey, chandler)",
    "best_friend(joey, ross)",
    "wife(phoebe, Husband)",
    "brother(X, monica)",
]

query_result = []
for q in queries:
    print(f"Query: {q}")
    results = list(prolog.query(q))
    for r in results:
        print(r)
    query_result.append((q,results))
    print()


Query: best_friend(joey, X)
{'X': 'chandler'}

Query: best_friend(joey, chandler)
{}

Query: best_friend(joey, ross)

Query: wife(phoebe, Husband)
{'Husband': 'mike'}

Query: brother(X, monica)
{'X': 'ross'}



In [24]:
query_result

[('best_friend(joey, X)', [{'X': 'chandler'}]),
 ('best_friend(joey, chandler)', [{}]),
 ('best_friend(joey, ross)', []),
 ('wife(phoebe, Husband)', [{'Husband': 'mike'}]),
 ('brother(X, monica)', [{'X': 'ross'}])]

### Convert the prolog result back to Natural Language

In [26]:
for query, result in query_result:
    prompt = f"""Convert the following Prolog query and result into a natural language answer. Print the query and result in the prompt, and then provide a concise answer.

            Query: {query}
            Result: {result}

            Answer:"""
    
    response = client.responses.create(
        model="gpt-4.1",
        input=prompt
    ).output_text

    print(response)
    print()


Query: best_friend(joey, X)  
Result: [{'X': 'chandler'}]

Answer: Joey's best friend is Chandler.

Query: best_friend(joey, chandler)  
Result: [{}]

Answer: Yes, Joey is best friends with Chandler.

Query: best_friend(joey, ross)  
Result: []

Answer: Joey and Ross are not best friends.

Query: wife(phoebe, Husband)  
Result: [{'Husband': 'mike'}]

Answer: Phoebe's husband is Mike.

Query: brother(X, monica)  
Result: [{'X': 'ross'}]

Answer: Ross is Monica's brother.

